In [63]:
import altair as alt # https://altair-viz.github.io/user_guide/interactions.html
alt.data_transformers.disable_max_rows()
import datapane as dp # https://www.youtube.com/watch?v=_KS_yZBI71s&ab_channel=Datapane
import pandas as pd

# Setup

There must be a csv file with the job and build sections duration. This can be generated using [parse_buildtimes.py](parse_buildtimes.py) script.

Example using two different jobs:

```bash
export d=build.ros2.org
export c=23
cd ~/buildfarm-tools/testdb
./fetch.rb -d $d -j Rci__nightly-release_ubuntu_noble_amd64 -c $c timestamps
./fetch.rb -d $d -j Rci__nightly-release_ubuntu_jammy_amd64 -c $c timestamps
cd ../scripts/metrics
./parse_buildtimes.py -d $d -j Rci__nightly-release_ubuntu_noble_amd64 -c $c -o noble_vs_jammy.csv
./parse_buildtimes.py -d $d -j Rci__nightly-release_ubuntu_jammy_amd64 -c $c -o noble_vs_jammy.csv --reuse
```

In [122]:
# Load data
buildtimes_df = pd.read_csv("ciros2_noble_vs_jammy.csv")
buildtimes_df["duration_seconds"] = buildtimes_df["duration_seconds"].apply(lambda x: x/60)
buildtimes_df.rename(columns={"duration_seconds": "duration_minutes"}, inplace=True)

# Set variables
job1 = "nightly_linux_release_noble"
job2 = "nightly_linux_release_jammy"
section_type = "section"
duration_threshold = 1 # minutes

# print(buildtimes_df[buildtimes_df["section_name"].str.contains(r"\[test\]")].to_markdown())
# Average build times for each section
buildtimes_df = (buildtimes_df
                 .query(f"job_name == '{job1}' or job_name == '{job2}'")
                 .query(f"section_type == '{section_type}'")
                 .groupby(["job_name", "section_name"])
                 .agg(duration_minutes=("duration_minutes", "mean"), sample_size=("duration_minutes", "count"), std_dev=("duration_minutes", "std"))
                 .reset_index()
            )

# Save for plotting
plot_df = buildtimes_df.copy(deep=True)

# # Calculate the difference in build times
diff_df = pd.merge(buildtimes_df[buildtimes_df["job_name"] == job1], buildtimes_df[buildtimes_df["job_name"] == job2], on="section_name", suffixes=("_noble", "_jammy"), how="inner")
diff_df.drop(columns=["job_name_noble", "job_name_jammy"], inplace=True)
diff_df["difference_minutes"] = diff_df["duration_minutes_noble"] - diff_df["duration_minutes_jammy"]

# Sort and filter
diff_df = (diff_df
           .round(3)
           .drop(diff_df[abs(diff_df.difference_minutes) < duration_threshold].index)
           .sort_values("difference_minutes", ascending=False)
           .reset_index(drop=True)
        )

# Print markdown
print(diff_df.to_markdown())

# Sort and filter out short build times
plot_df = (plot_df
           .drop(plot_df[abs(plot_df.duration_minutes) < duration_threshold].index)
        )

# Plot
alt.Chart(plot_df).mark_bar().encode(
    x=alt.X('job_name', title='Job Name', axis=None),
    y=alt.Y('duration_minutes'),
    color='job_name',
    column=alt.Column('section_name', title='Section name', header=alt.Header(labelOrient='top', labelAngle=90, labelAnchor='start'))
)

|    | section_name     |   duration_minutes_noble |   sample_size_noble |   std_dev_noble |   duration_minutes_jammy |   sample_size_jammy |   std_dev_jammy |   difference_minutes |
|---:|:-----------------|-------------------------:|--------------------:|----------------:|-------------------------:|--------------------:|----------------:|---------------------:|
|  0 | Build Dockerfile |                    1.396 |                  17 |           1.865 |                    0.361 |                  54 |           0.752 |                1.035 |
|  1 | Run Dockerfile   |                   75.211 |                  16 |          16.201 |                   81.721 |                  51 |          27.664 |               -6.51  |


alt.Chart(...)

# Results ci.ros2.org noble vs jammy

csv data from `ciros2_noble_vs_jammy.csv`:
* Contains only builds that are run in linux-2xlarge machines
* Builds from noble are taken since 2024-04-17
* Builds from jammy are taken before that date
* Sample size is the last 100 builds from each job


Noble since build:
* Debug: 3029
* Release: 3045
* Repeated: 3427

<details>
<summary>
<h3>Sections</h3>
</summary>

#### Debug

| section_name     |   duration_minutes_noble |   duration_minutes_jammy |   difference_minutes |
|:-----------------|-------------------------:|-------------------------:|---------------------:|
| Build Dockerfile |                    2.531 |                    0.738 |                1.792 |
| Run Dockerfile   |                   73.644 |                   86.502 |              -12.858 |

#### Release

| section_name     |   duration_minutes_noble |   duration_minutes_jammy |   difference_minutes |
|:-----------------|-------------------------:|-------------------------:|---------------------:|
| Build Dockerfile |                    1.396 |                    0.361 |                1.035 |
| Run Dockerfile   |                   75.211 |                   81.721 |               -6.51  |

#### Repeated

| section_name   |   duration_minutes_noble |   duration_minutes_jammy |   difference_minutes |
|:---------------|-------------------------:|-------------------------:|---------------------:|
| Run Dockerfile |                  121.206 |                  145.423 |              -24.217 |


</details>

<details>
<summary>
<h3>Subsections</h3>
</summary>

#### Debug

| section_name           |   duration_minutes_noble |   duration_minutes_jammy |   difference_minutes |
|:-----------------------|-------------------------:|-------------------------:|---------------------:|
| build [Run Dockerfile] |                   40.133 |                   37.688 |                2.445 |
| test [Run Dockerfile]  |                   30.507 |                   45.035 |              -14.528 |


#### Release

| section_name           |   duration_minutes_noble |   duration_minutes_jammy |   difference_minutes |
|:-----------------------|-------------------------:|-------------------------:|---------------------:|
| build [Run Dockerfile] |                   44.19  |                   36.624 |                7.567 |
| test [Run Dockerfile]  |                   29.311 |                   42.673 |              -13.362 |

#### Repeated

| section_name           |   duration_minutes_noble |   duration_minutes_jammy |   difference_minutes |
|:-----------------------|-------------------------:|-------------------------:|---------------------:|
| build [Run Dockerfile] |                   36.161 |                   37.634 |               -1.474 |
| test [Run Dockerfile]  |                   80.435 |                  102.813 |              -22.378 |


</details>


<details>
<summary>
<h3>Packages</h3>
</summary>

#### Debug

| section_name                    |   duration_minutes_noble |   duration_minutes_jammy |   difference_minutes |
|:--------------------------------|-------------------------:|-------------------------:|---------------------:|
| std_msgs [build]                |                    3.524 |                    0.385 |                3.139 |
| geometry_msgs [build]           |                    2.88  |                    0.924 |                1.956 |
| robot_state_publisher [build]   |                    2.995 |                    1.233 |                1.762 |
| turtlesim [build]               |                    3.559 |                    2.354 |                1.206 |
| test_quality_of_service [build] |                    3.714 |                    2.647 |                1.067 |
| rviz_default_plugins [build]    |                    7.696 |                    9.061 |               -1.364 |
| test_communication [build]      |                    0.672 |                   16.614 |              -15.943 |


#### Release

| section_name                  |   duration_minutes_noble |   duration_minutes_jammy |   difference_minutes |
|:------------------------------|-------------------------:|-------------------------:|---------------------:|
| fastrtps [build]              |                   12.113 |                    4.451 |                7.662 |
| rviz_ogre_vendor [build]      |                    8.872 |                    4.18  |                4.692 |
| example_interfaces [build]    |                    3.865 |                    1.905 |                1.961 |
| rviz_common [build]           |                    5.176 |                    3.274 |                1.902 |
| rviz_rendering [build]        |                    2.949 |                    1.235 |                1.714 |
| test_rclcpp [build]           |                    4.12  |                    2.642 |                1.478 |
| point_cloud_transport [build] |                    3.152 |                    1.708 |                1.444 |
| test_communication [build]    |                    5.433 |                    4.269 |                1.164 |
| test_tracetools [build]       |                    0.283 |                    1.539 |               -1.256 |


#### Repeated

| section_name                    |   duration_minutes_noble |   duration_minutes_jammy |   difference_minutes |
|:--------------------------------|-------------------------:|-------------------------:|---------------------:|
| rviz_default_plugins [build]    |                    6.605 |                    8.101 |               -1.496 |
| rviz_common [build]             |                    4.555 |                    6.159 |               -1.604 |
| test_quality_of_service [build] |                    1.552 |                    3.221 |               -1.67  |
| cyclonedds [build]              |                    1.426 |                    3.104 |               -1.678 |
| test_communication [build]      |                    5.058 |                   11.603 |               -6.546 |
| demo_nodes_cpp [build]          |                    0.41  |                    9.238 |               -8.828 |


</details>


# Buildtime Analysis

sql query used to generate the data:

```sql
SELECT job_name, build_number, buildtime/60000.0 as buildtime FROM build_status
WHERE node_name LIKE "linux-2xlarge-%" AND status != 'FAILURE'  
    AND job_name == 'nightly_linux_release' AND build_number BETWEEN 2971 AND 3070
ORDER BY build_number DESC;
```

For the first builds of a node:
```sql
SELECT job_name, build_number, buildtime/60000.0 as buildtime, node_name, min(build_datetime) FROM build_status
WHERE node_name LIKE "linux-2xlarge-%" AND status != 'FAILURE'
	-- AND job_name == 'nightly_linux_release' AND build_number BETWEEN 2969 AND 3073
	-- AND job_name == 'nightly_linux_debug' AND build_number BETWEEN 2954 AND 3053
	AND job_name == 'nightly_linux_repeated' AND build_number BETWEEN 3352 AND 3451
GROUP BY node_name
ORDER BY build_number DESC;
```

In [132]:
job_builtimes = pd.read_csv("ciros2_release_buildtime.csv")
first_builds = pd.read_csv("ciros2_noble_vs_jammy_first_build.csv")

job1 = "nightly_linux_repeated_noble"
job2 = "nightly_linux_repeated_jammy"

job1_normal_buildtimes = job_builtimes.query(f"job_name == '{job1}' and build_number not in {list(first_builds['build_number'])}", engine="python")
job1_normal_builtime_avg = job1_normal_buildtimes["duration_minutes"].mean()
job1_normal_buildtime_std = job1_normal_buildtimes["duration_minutes"].std()
job2_normal_buildtimes = job_builtimes.query(f"job_name == '{job2}' and build_number not in {list(first_builds['build_number'])}", engine="python")
job2_normal_buildtime_avg = job2_normal_buildtimes["duration_minutes"].mean()
job2_normal_buildtime_std = job2_normal_buildtimes["duration_minutes"].std()


job1_first_buildtimes = first_builds.query(f"job_name == '{job1}'")
job1_first_buildtimes_avg = job1_first_buildtimes["duration_minutes"].mean()
job1_first_buildtimes_std = job1_first_buildtimes["duration_minutes"].std()
job2_first_buildtimes = first_builds.query(f"job_name == '{job2}'")
job2_first_buildtimes_avg = job2_first_buildtimes["duration_minutes"].mean()
job2_first_buildtimes_std = job2_first_buildtimes["duration_minutes"].std()

print(f"| {job1} | {job1_normal_builtime_avg:.3f} | {job1_first_buildtimes_avg:.3f} | {job1_first_buildtimes_avg - job1_normal_builtime_avg:.3f} |")
print(f"| {job2} | {job2_normal_buildtime_avg:.3f} | {job2_first_buildtimes_avg:.3f} | {job2_first_buildtimes_avg - job2_normal_buildtime_avg:.3f} |")

# Check buildtimes of agent's first run
# Maybe check timeouts in build.ros2.org

| nightly_linux_repeated_noble | 108.569 | 131.699 | 23.130 |
| nightly_linux_repeated_jammy | 135.005 | 154.870 | 19.865 |



### Differentiating between node first builds and normal builds

| Job Name | Average Normal Buildtime | Average Node First Buildtime | Difference |
|:-------|-----------------------------:|------------------------------:|------------:|
| nightly_linux_debug (noble) | 55.284 | 88.766 | 33.482 |
| nightly_linux_debug (jammy) | 83.897 | 99.662 | 15.765 |
| nightly_linux_release (noble) | 63.951 | 84.653 | 20.701 |
| nightly_linux_release (jammy) | 76.238 | 94.847 | 18.609 |
| nightly_linux_repeated (noble) | 108.569 | 131.699 | 23.130 |
| nightly_linux_repeated (jammy) | 135.005 | 154.870 | 19.865 |



### All builds considered
| Job Name | Average Buildtime (minutes) | Standard Deviation (minutes) | Sample Size |
|:-------|-----------------------------:|------------------------------:|------------:|
| nightly_linux_repeated (noble) | 123.535 | 18.144 | 17 |
| nightly_linux_repeated (jammy) | 147.016 | 41.882 | 43 |
| nightly_linux_debug (noble) | 77.377 | 19.503 | 24 |
| nightly_linux_debug (jammy) | 88.076 | 29.931 | 71 |
| nightly_linux_release (noble) | 77.841 | 17.089 | 16 |
| nightly_linux_release (jammy) | 83.012 | 27.631 | 51 |

